In [1]:
import logging
import os
import pickle

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

from s3dis_util import crop_pc
from s3dis import S3DIS
import s3dis_transforms as T
import pickle
from models.model_semseg import Net

In [10]:
train_ds1 = S3DIS(split='train', voxel_max=24000, presample=True, transform=T.Compose([
    T.AppendHeight(), T.PointCloudFloorCentering(), T.RandomScale(),
    T.RandomRotate(), T.RandomJitter(),
    T.ChromaticAutoContrast(),
    T.RandomDropColor(),
    T.ToTensor()
]))
test_ds = S3DIS(split='val', transform=T.Compose([
    T.AppendHeight(), T.PointCloudFloorCentering(),
    T.ChromaticNormalize(), T.ToTensor()])
)

data/S3DIS/s3disfull/processed/s3dis_train_area5_0.040.pkl load successfully


In [11]:
train_ds2 = S3DIS(split='train', voxel_max=24000, presample=True, transform=T.Compose([
    T.AppendHeight(), T.PointCloudFloorCentering(), T.RandomScale(),
    T.RandomRotate(), T.RandomJitter(),
    T.ChromaticAutoContrast(),
    T.ToTensor()
]))


data/S3DIS/s3disfull/processed/s3dis_train_area5_0.040.pkl load successfully


In [12]:
with open("args.pkl", "rb") as f:
    args = pickle.load(f)

In [13]:
# model = Net(args).cuda(3)

In [14]:
dl1 = DataLoader(train_ds1, batch_size=2, shuffle=False)
dl2 = DataLoader(train_ds2, batch_size=2, shuffle=False)


In [15]:
for d1, d2 in zip(dl1, dl2):
    c1, f1, l1 = d1.values()
    c2, f2, l2 = d2.values()

    print(f1[0, :, 3])
    print('=======')
    print(f2[0, :, 3])

    break

tensor([0.9920, 1.4310, 1.4350,  ..., 0.1470, 1.0170, 1.2580])
tensor([0.9920, 1.4310, 1.4350,  ..., 0.1470, 1.0170, 1.2580])


In [6]:
raw_root = 'data/S3DIS/s3disfull/raw/'
test_area = '5'
data_list = sorted(os.listdir(raw_root))
data_list = [item[:-4] for item in data_list if 'Area_' in item]
data_list = [item for item in data_list if 'Area_{}'.format(test_area) in item]


In [7]:
len(data_list)

68

In [8]:
data_idx = 0
data_path = os.path.join(raw_root, data_list[data_idx] + '.npy')
data_path


'data/S3DIS/s3disfull/raw/Area_5_WC_1.npy'

In [9]:
cdata = np.load(data_path).astype(np.float32)
cdata[:, :3] -= np.min(cdata[:, :3], 0)
coord, feat, label = cdata[:, :3], cdata[:, 3:6], cdata[:, 6:7]
print(coord.shape)


(719348, 3)


In [10]:
coord, feat, label = crop_pc(coord, feat, label, 'test', 0.04, None, downsample=True, variable=False)
coord.shape

(47355, 3)

In [11]:
label = label.squeeze(-1).astype(np.int_)
data = {'pos': coord, 'x': feat, 'y': label}


In [12]:
transform = T.Compose([
    T.PointCloudFloorCentering(),
    T.ChromaticNormalize(), T.ToTensor()])

In [13]:

data['pos'], data['x'], data['y'] = transform(
    data['pos'], data['x'], data['y'])
data['x'] = torch.cat((data['x'], torch.from_numpy(
coord[:, 3-1:3].astype(np.float32))), dim=-1)

In [14]:
data['pos'].shape, data['x'].shape

(torch.Size([47355, 3]), torch.Size([47355, 4]))

In [15]:
coord = data['pos'].transpose(0,1).unsqueeze(0).cuda(3)
feat = data['x'].transpose(0,1).unsqueeze(0).cuda(3)
coord.shape, feat.shape

(torch.Size([1, 3, 47355]), torch.Size([1, 4, 47355]))

In [16]:
with torch.no_grad():
    result = model(coord, feat)


RuntimeError: CUDA out of memory. Tried to allocate 8.36 GiB (GPU 3; 23.70 GiB total capacity; 16.72 GiB already allocated; 5.46 GiB free; 16.74 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF